In [2]:
import openai
from dotenv import find_dotenv, load_dotenv
import os
from fuzzywuzzy import fuzz
import random
import pandas as pd
import nltk.tokenize 

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

/Users/xlim/anaconda3/envs/gai_srch_summarization_p39/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/xlim/anaconda3/envs/gai_srch_summarization_p39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

PRODUCT_SUMMARY__PROMPT = """
Product Details:
{product_details}

Based on the context provided, generate a detailed paragraph summarizing the product with the following information:
- What does this product do?
- What are the features of this product?
- Who is the target audience for this product?
"""

In [4]:
#functions
def get_gpt_response(prompt, temperature=1.0, model = "gpt-3.5-turbo"):
    CHAT_COMPLETION_MODEL = model

    CHAT_COMPLETION_API_PARAMS = {
        # use temperature of 0.0 to give the most predictable, factual answer.
        # tweak temperature based on your use case
        "temperature": temperature,
        "model": CHAT_COMPLETION_MODEL,
    }

    messages = [
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
        messages=messages, **CHAT_COMPLETION_API_PARAMS
    )

    response_content = response["choices"][0]["message"]["content"]

    return response, response_content

def read_file_content(file_path):
    """
    read the content of the context files
    """
    with open(file_path, 'r') as file:
        file_content = file.read()
    return file_content



In [5]:
df_product_summary = pd.DataFrame(columns=['product_name', 'product_summary'])
# for each txt file in the context_store folder, call the summarization_main function
for file in os.listdir("../context_store"):
    if file.endswith(".txt"):
        print(file)
        file_content = read_file_content(os.path.join("../context_store", file))
        prompt = PRODUCT_SUMMARY__PROMPT.format(product_details=file_content)
        response, response_content = get_gpt_response(prompt)
        print(response_content)
        print('length_of_response: ', len(response_content))
        #store into df
        df_data = pd.DataFrame({'product_name': [file], 'product_summary': [response_content]})
        df_product_summary = pd.concat([df_product_summary, df_data], ignore_index=True)

roarmoney.txt
RoarMoney is a mobile bank account that aims to empower cash-strapped, tech-savvy millennials to take control of their finances. It offers a comprehensive banking solution with no hidden fees, providing users with physical and digital debit cards, the ability to get paid up to 2 days early and access up to $1,000 in interest-free cash advances. The account also offers instant money movement and rewards on spending. The target audience for this product is millennials living paycheck-to-paycheck, often working in the gig economy, and experiencing fluctuations in their income. These individuals are looking for ways to manage their money effectively and break free from the vicious cycle of financial insecurity. RoarMoney aims to provide them with affordable banking services and tools to build financial power and wealth.
length_of_response:  827
wow_membership.txt
MoneyLion WOW is a membership that offers users a premium experience with exclusive benefits across a range of pro

In [6]:
df_product_summary.to_csv('../result_files/product_summary.csv', index=False)